# Tweets

## 1 - Import useful modules

In [46]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split

## 2 - Load and prepare dataset

In [47]:
regex = re.compile(r',\b')
with open('data/processedPositive.csv') as f:
    pos_data = regex.sub('|', f.read()).split('|')
with open('data/processedNegative.csv') as f:
    neg_data = regex.sub('|', f.read()).split('|')
with open('data/processedNeutral.csv') as f:
    neu_data = regex.sub('|', f.read()).split('|')

processed_positive_df = pd.DataFrame({'tweets': pos_data, 'labels': 1})
processed_negative_df = pd.DataFrame({'tweets': neg_data, 'labels': -1})
processed_neutral_df = pd.DataFrame({'tweets': neu_data, 'labels': 0})

Concatenate all three categories into one dataframe.

In [48]:
df = pd.concat([processed_positive_df, processed_negative_df, processed_neutral_df])

Split the dataset into 80% training and 20% test with stratification.

In [50]:
X_train, X_test, y_train, y_test = train_test_split(df['tweets'], df['labels'], test_size=0.2, stratify = df['labels'], random_state=1337)